# Coordinate Attention Güçlendirme Eklentileri 
Bu notebook, **mevcut CoordAtt koduna** ekleyebileceğin parçaları verir. Matematik yok; her ek için:
- **Ne eklenir (kod)**
- **Ne işe yarar (amaç)**
- **Model neden güçlenir (etki)**


## 0) Başlangıç: Referans CoordAtt (temel)
Aşağıdaki sınıf, eklentilerin üzerine oturtulacağı **temel** sürümdür. Eklentiler bu yapıyı bozmadan güçlendirmeyi hedefler.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HSwish(nn.Module):
    def forward(self, x):
        return x * F.relu6(x + 3.0, inplace=True) / 6.0

class CoordinateAtt(nn.Module):
    def __init__(
        self,
        in_channels: int,
        reduction: int = 32,
        min_mid_channels: int = 8,
        act: str = "hswish",
        alpha: float = 1.0,
        learnable_alpha: bool = False,
    ):
        super().__init__()
        mid_channels = max(min_mid_channels, in_channels // reduction)

        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)

        if act.lower() == "hswish":
            self.act = HSwish()
        elif act.lower() == "relu":
            self.act = nn.ReLU(inplace=True)
        else:
            raise ValueError("act must be 'hswish' or 'relu'")

        self.conv_h = nn.Conv2d(mid_channels, in_channels, kernel_size=1, bias=True)
        self.conv_w = nn.Conv2d(mid_channels, in_channels, kernel_size=1, bias=True)

        if learnable_alpha:
            self.alpha = nn.Parameter(torch.tensor(float(alpha)))
        else:
            self.register_buffer("alpha", torch.tensor(float(alpha)))

        self._last_ah = None
        self._last_aw = None

    def forward(self, x):
        B, C, H, W = x.shape

        x_h = x.mean(dim=3, keepdim=True)                    # (B,C,H,1)
        x_w = x.mean(dim=2, keepdim=True).permute(0,1,3,2)   # (B,C,W,1)

        y = torch.cat([x_h, x_w], dim=2)                     # (B,C,H+W,1)
        y = self.act(self.bn1(self.conv1(y)))                # (B,mid,H+W,1)

        y_h, y_w = torch.split(y, [H, W], dim=2)             # (B,mid,H,1) & (B,mid,W,1)
        y_w = y_w.permute(0,1,3,2)                           # (B,mid,1,W)

        a_h = torch.sigmoid(self.conv_h(y_h))                # (B,C,H,1)
        a_w = torch.sigmoid(self.conv_w(y_w))                # (B,C,1,W)

        self._last_ah = a_h
        self._last_aw = a_w

        att = a_h * a_w                                      # (B,C,H,W)
        scale = (1.0 - self.alpha) + self.alpha * att

        return x * scale

    @torch.no_grad()
    def last_mask_stats(self):
        if self._last_ah is None or self._last_aw is None:
            return None
        ah = self._last_ah
        aw = self._last_aw
        return {
            "a_h": {"min": float(ah.min()), "mean": float(ah.mean()), "max": float(ah.max()), "std": float(ah.std())},
            "a_w": {"min": float(aw.min()), "mean": float(aw.mean()), "max": float(aw.max()), "std": float(aw.std())},
        }


## 1) Multi-Scale Axis Pooling (Mean + Max)
**Amaç:** Sadece ortalama almak bazen ayrıntıyı düzleştirir. Mean+Max ile eksen özetleri daha zengin olur.
**Model neden güçlenir:** Maskeler daha ayırt edici çıkar; zayıf ama önemli aktivasyonlar kaybolmaz.

**Ne eklenir:** forward içinde `x_h` ve `x_w` hesaplarını aşağıdaki gibi değiştir.

In [ ]:
# Eski:
# x_h = x.mean(dim=3, keepdim=True)
# x_w = x.mean(dim=2, keepdim=True).permute(0,1,3,2)

# Yeni (Mean + Max):
x_h_mean = x.mean(dim=3, keepdim=True)
x_h_max  = x.amax(dim=3, keepdim=True)
x_h = 0.5 * (x_h_mean + x_h_max)                           # (B,C,H,1)

x_w_mean = x.mean(dim=2, keepdim=True)
x_w_max  = x.amax(dim=2, keepdim=True)
x_w = 0.5 * (x_w_mean + x_w_max)                           # (B,C,1,W)
x_w = x_w.permute(0, 1, 3, 2)                              # (B,C,W,1)


## 2) Axis-wise Depthwise Conv (Lokal eksen bilgisi)
**Amaç:** Eksen özetleri sadece global istatistik olmasın; eksen boyunca lokal süreklilik de görülsün.
**Model neden güçlenir:** İnce konum farkları daha iyi yakalanır; maskeler daha stabil olur.

**Ne eklenir:** `__init__` içine iki depthwise katman, `forward` içinde x_h/x_w üzerine uygula.

In [ ]:
# __init__ içine ekle:
self.dw_h = nn.Conv2d(in_channels, in_channels, kernel_size=(3,1), padding=(1,0), groups=in_channels, bias=False)
self.dw_w = nn.Conv2d(in_channels, in_channels, kernel_size=(1,3), padding=(0,1), groups=in_channels, bias=False)

# forward içinde (x_h ve x_w hesaplandıktan sonra):
x_h = self.dw_h(x_h)                      # (B,C,H,1)

x_w_1w = x_w.permute(0,1,3,2)             # (B,C,1,W)
x_w_1w = self.dw_w(x_w_1w)                # (B,C,1,W)
x_w = x_w_1w.permute(0,1,3,2)             # (B,C,W,1)


## 3) Dilated Axis Conv (Daha geniş bağlam)
**Amaç:** Eksen boyunca daha uzak ilişkileri de kapsamak.
**Model neden güçlenir:** Büyük objelerde/uzun yapılarlarda daha tutarlı attention çıkar.

**Ne eklenir:** 2) maddesindeki depthwise conv’ları dilated yap (yalnızca parametrelerle).

In [ ]:
# __init__ içinde (örnek: dilation=2):
d = 2
self.dw_h = nn.Conv2d(in_channels, in_channels, kernel_size=(3,1), padding=(d,0), dilation=(d,1),
                      groups=in_channels, bias=False)
self.dw_w = nn.Conv2d(in_channels, in_channels, kernel_size=(1,3), padding=(0,d), dilation=(1,d),
                      groups=in_channels, bias=False)


## 4) Eksen Bazlı Alpha (alpha_h, alpha_w)
**Amaç:** Dikey ve yatay maskelerin etkisini ayrı kontrol etmek.
**Model neden güçlenir:** Stabilite artar; bir eksen hatalıysa diğerini daha az etkiler.

**Ne eklenir:** tek `alpha` yerine iki alpha. `scale` hesaplaması değişir.

In [ ]:
# __init__ içinde tek alpha yerine:
if learnable_alpha:
    self.alpha_h = nn.Parameter(torch.tensor(float(alpha)))
    self.alpha_w = nn.Parameter(torch.tensor(float(alpha)))
else:
    self.register_buffer("alpha_h", torch.tensor(float(alpha)))
    self.register_buffer("alpha_w", torch.tensor(float(alpha)))

# forward sonunda:
scale_h = (1.0 - self.alpha_h) + self.alpha_h * a_h         # (B,C,H,1)
scale_w = (1.0 - self.alpha_w) + self.alpha_w * a_w         # (B,C,1,W)
scale = scale_h * scale_w                                   # (B,C,H,W)
return x * scale


## 5) Sigmoid yerine Hard-Sigmoid (Daha stabil kapı)
**Amaç:** Maskeler çok erken 0/1’e yapışmasın.
**Model neden güçlenir:** Grad akışı daha stabil olur; maskeler ‘ölmez’.

**Ne eklenir:** `torch.sigmoid` yerine `F.hardsigmoid`.

In [ ]:
# Eski:
# a_h = torch.sigmoid(self.conv_h(y_h))
# a_w = torch.sigmoid(self.conv_w(y_w))

# Yeni:
a_h = F.hardsigmoid(self.conv_h(y_h), inplace=True)
a_w = F.hardsigmoid(self.conv_w(y_w), inplace=True)


## 6) Attention Residual (Maske kapatmasın)
**Amaç:** Maskeler 0’a yaklaşınca sinyalin tamamen kapanmasını engellemek.
**Model neden güçlenir:** Over-suppression azalır; özellikle derin ağlarda daha güvenli.

**Ne eklenir:** `att` üzerine küçük residual form (isteğe bağlı beta).

In [ ]:
# __init__ içine (isteğe bağlı):
self.beta = 0.5  # 0.2–1.0 aralığında denenebilir (learnable da yapılabilir)

# forward sonunda:
att = a_h * a_w
att = 1.0 + self.beta * (att - 1.0)
scale = (1.0 - self.alpha) + self.alpha * att
return x * scale


## 7) Mid Katmanını Güçlendirme (2 katmanlı bottleneck)
**Amaç:** Ortak latent uzayın ifade gücünü artırmak.
**Model neden güçlenir:** Maskeler daha ‘anlamlı’ çıkar; çok az ek maliyetle kapasite artar.

**Ne eklenir:** `conv1` tek katman yerine iki 1×1 katman (mid içinde).

In [ ]:
# __init__ içinde:
self.conv1a = nn.Conv2d(in_channels, mid_channels, kernel_size=1, bias=False)
self.bn1a   = nn.BatchNorm2d(mid_channels)
self.conv1b = nn.Conv2d(mid_channels, mid_channels, kernel_size=1, bias=False)
self.bn1b   = nn.BatchNorm2d(mid_channels)

# forward içinde:
y = self.act(self.bn1a(self.conv1a(y)))
y = self.act(self.bn1b(self.conv1b(y)))


## 8) Head’leri Hafifletme (Grouped 1×1)
**Amaç:** Parametre ve hesap maliyetini düşürmek.
**Model neden güçlenir:** Aynı bütçede daha büyük backbone mümkün olur (pratik kazanç).

**Ne eklenir:** `conv_h/conv_w` için groups>1.

In [ ]:
# __init__ içinde (in_channels % g == 0 olmalı):
g = 4
self.conv_h = nn.Conv2d(mid_channels, in_channels, kernel_size=1, groups=g, bias=True)
self.conv_w = nn.Conv2d(mid_channels, in_channels, kernel_size=1, groups=g, bias=True)


## 9) Mini Spatial Gate (Çok hafif 2D düzeltme)
**Amaç:** CoordAtt eksensel çalışır; bazen küçük bir 2D düzeltme faydalıdır.
**Model neden güçlenir:** İnce 2D konum hassasiyeti artar; CBAM-SA kadar ağır olmadan kazanım sağlar.

**Ne eklenir:** CoordAtt çıkışı sonrası küçük depthwise 3×3 gate.

In [ ]:
# __init__ içine:
self.spatial_dw = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, groups=in_channels, bias=False)
self.spatial_pw = nn.Conv2d(in_channels, in_channels, kernel_size=1, bias=True)

# forward sonunda (return’dan önce):
out = x * scale
gate = torch.sigmoid(self.spatial_pw(self.spatial_dw(out)))
return out * gate


## 10) Debug: Maske çökmesi var mı?
**Amaç:** Maske dağılımı aşırı daralıyor mu (std çok küçük) veya saturasyon var mı görmek.
**Model neden güçlenir:** Hızlı teşhis → doğru eklentiyi seçersin (örn. hard-sigmoid, alpha düşürme).

**Ne eklenir:** Hızlı kontrol fonksiyonu.

In [ ]:
@torch.no_grad()
def quick_mask_check(att_module, x):
    att_module.eval()
    _ = att_module(x)
    stats = att_module.last_mask_stats() if hasattr(att_module, "last_mask_stats") else None
    print(stats)
    if stats is not None:
        if stats["a_h"]["std"] < 0.02 or stats["a_w"]["std"] < 0.02:
            print("UYARI: maske std çok düşük -> maske sabitleniyor olabilir.")
